In [1]:
# Project Template Import Cell
%matplotlib inline
import os
import sys
import math

from __future__ import print_function, division
from inspect import getsourcefile

from IPython.display import FileLink

# Standard Notebook Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

CURR_PATH = os.path.abspath(getsourcefile(lambda:0)).rsplit('/', 1)[0] # Get filepath of this notebook
module_path = os.path.join(os.path.dirname(CURR_PATH), 'utils') # Make module path for one dir up and one down into utils
if module_path not in sys.path: # Append to system path list
    sys.path.append(module_path)

import utils as utils ; reload(utils)

print('System : Python ', os.sys.version , '\n\n')

print('Directory Structure:')
print(CURR_PATH)
!tree -cn --filelimit 16

# Keras Setup
import keras
print('\n\nKeras version:' , keras.__version__ ,
      ', backend:' , keras.backend.backend(),
      ', image_format:' , keras.backend.image_data_format())

random_seed = 2

print('\n\nEnvironment : ')
FileLink('../../ana_py27_p2.yml')

Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


System : Python  2.7.12 | packaged by conda-forge | (default, Sep  8 2016, 14:22:31) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)] 


Directory Structure:
/home/ubuntu/ana_py27_p2/projects/CervicalCancer
.
├── unused_utils
│   ├── __init__.py
│   ├── __init__.pyc
│   ├── utils.pyc
│   └── unused_utils.py
├── __init__.py
├── 001 - Initial Setup.ipynb
├── 002 - Data Setup & General Notes.ipynb
├── 003 - Baseline.ipynb
├── Trash
│   ├── 002-01 - Train Data (only) Setup.ipynb
│   └── Base CNN Model Train Data with Preprocessing.ipynb
├── 004-1 - Base CNN Model - Sample.ipynb
├── 004-2 - Base CNN Model - Train Data.ipynb
├── 005 - Exploratoring the Data.ipynb
├── 006 - Non Neural Net Classifiers.ipynb
├── 007 - Gradient Boosting Classifier.ipynb
├── data
│   ├── additional_Type_1_v2.7z
│   ├── additional_Type_3_v2.7z
│   ├── additional_Type_2_v2.7z
│   ├── train
│   │   ├── Type_1 [168 entries exceeds filelimit, not opening dir]
│   │   ├── Type_2 [524 entries exceeds filelimit, not opening dir]

/home/ubuntu/ana_py27_p2/ana_py27_p2.yml

### My Strategy
[Fine-tuning the top layers of a a pre-trained network](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html) - I will attempt to follow the Final Part of this tutorial, but the final part build upon work done in Part 2, so I will start at Part 2 - Using the bottleneck features of a pre-trained network: 90% accuracy in a minute

1. Load up VGG16 and its weights
2. Chop off the top layer of VGG16, and add a small fully connected model
3. Freeze all the layers in VGG16, and train the final fully connected model
4. Unfreeze the last convolutional layer also, and retrain the model

### Housekeeping

In [2]:
# A fix for truncated image error
# http://stackoverflow.com/questions/12984426/python-pil-ioerror-image-file-truncated-with-big-images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# DIRECTORIES
# !!!SAMPLE
train_data_dir = 'data/sample/train' ; validation_data_dir = 'data/sample/valid' ; test_data_dir = 'data/sample/test'
weights_dir = 'data/sample/weights'

# !!!TRAIN
#train_data_dir = 'data/train' ; validation_data_dir = 'data/valid' ; test_data_dir = 'data/test'
#weights_dir = 'data/weights'

submission_dir = 'data/submissions'
pt_models_dir = '../pretrained_models'
models_dir = 'data/models'

In [3]:
# DATA META-DATA
# Get the DataSets and load into a DataFrame for easy reference

img_width, img_height = 224, 224
#img_width, img_height = 300, 300

num_training_samples = 0
for class_dir in utils.get_non_hidden_dir_contents(train_data_dir):
    num_training_samples += len(utils.get_non_hidden_dir_contents(class_dir))
num_validation_samples = 0
for class_dir in utils.get_non_hidden_dir_contents(validation_data_dir):
    num_validation_samples += len(utils.get_non_hidden_dir_contents(class_dir))    

print(num_training_samples)
print(num_validation_samples)

# !!!SAMPLE
epochs = 15
batch_size=64

# !!!TRAIN
#epochs=30
#batch_size=64

input_shape = (img_width, img_height, 3)

train_steps = int(math.ceil(num_training_samples / batch_size))
validation_steps =  int(math.ceil(num_validation_samples / batch_size))
print(train_steps)
print(validation_steps)

473
362
8
6


### Model Building

In [4]:
# Load up VGG16 architecture & inspect
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout
from keras.models import model_from_yaml

import yaml
# Load the full vgg16 model
yaml_file = open(os.path.join(pt_models_dir, 'vgg16model_1.yaml'), 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
vgg16d_1 = model_from_yaml(loaded_model_yaml)

# Load the original vgg16 model weights
vgg16d_1.load_weights(os.path.join(pt_models_dir,'vgg16weights_1.hdf5'))
print("Loaded weights from disk")

#vgg16.summary()
# Chop off the last 12 layers- the last dense block, the last 2 conv blocks
#vgg16_p = Model(inputs=[vgg16.input], outputs=[vgg16.get_layer('block5_pool').output]) #Grab output by name
vgg16d_1 = Model(inputs=[vgg16d_1.input], outputs=[vgg16d_1.layers[-9].output])
#utils.show_ascii_model(vgg16_p)
from keras import regularizers
# Create a new Dense model to add to the end of vgg16
x = Flatten()(vgg16d_1.layers[-1].output)
x = Dense(256, activation='relu') (x)

#x = Dense(256, activation='relu', kernel_regularizer= regularizers.l2(1.0)) (x)

x = Dropout(0.7) (x)

last = Dense(3, activation='softmax')(x)

# Add to the end of VGG16
vgg16d_1 = Model(inputs=[vgg16d_1.input], outputs=[last])
# load weights into new model


#utils.show_ascii_model(vgg16_)
#Freeze all the weights in the original vgg16 layers before compile
for l in vgg16d_1.layers:
    l.trainable=False
for l in vgg16d_1.layers[10:]:
    l.trainable = True
vgg16d_1.summary() # Trainable parameters about 13,502,979 i.e 13 million compared to the original 138 million
# We can actually only train the weights in the final convolution block, and in the final dense block
#print(vgg16d_1.layers)

Loaded weights from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)   

In [5]:
#stop

In [6]:
from keras.optimizers import SGD
sgd = SGD(lr=1e-6, momentum=0.9) #

In [7]:
vgg16d_1.compile(loss='categorical_crossentropy',
              optimizer=sgd,
            metrics=['accuracy'])

In [8]:
#https://github.com/Lasagne/Recipes/issues/20
# https://stats.stackexchange.com/questions/217481/how-to-normalize-data-for-vgg-16-pretrained-model
def preprocess(img):
    """
    I don't get this - I thought this took an individual image
    I am missing a dimension somewhere
    Does it take a batch of images?? - I don't think so according to docs
    ???????????????????
    """
    #print(img.shape) #(224, 224, 3)
    img = np.expand_dims(img, axis=0)
    #print(img.shape) #(1, 224, 224, 3)
    # 'RGB'->'BGR'
    img = img[:, :, :, ::-1]
    # Zero-center by mean pixel
    img[:, :, :, 0] -= 103.939
    img[:, :, :, 1] -= 116.779
    img[:, :, :, 2] -= 123.68
    #img[:, :, :, 0] -= 78.939
    #img[:, :, :, 1] -= 91.779
    #img[:, :, :, 2] -= 148.68
    #print(img.shape)
    #print(img)
    return img

import cv2
import PIL
#import PIL.Image
from keras.preprocessing.image import array_to_img, img_to_array, load_img
def preprocess_pk(img):
    """
    I don't get this - I thought this took an individual image
    I am missing a dimension somewhere
    Does it take a batch of images?? - I don't think so according to docs
    ???????????????????
    """
    # Set parameters
    min_params = (130,100,150) ; max_params = (180,256,256)
    # Convert back into a PIL Image
    pil_img = array_to_img(img)
    # Convert into a cv2 image
    cv_img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    # Convert color space to HSV
    hsv = cv2.cvtColor(cv_img, cv2.COLOR_BGR2HSV)
    # Create mask by applying thresholds
    mask = cv2.inRange(hsv, min_params, max_params)
    i, j = np.where(mask)
    indices=None
    if any(i) or any(j):
        indices = np.meshgrid(np.arange(min(i), max(i) + 1),
                              np.arange(min(j), max(j) + 1),
                              indexing='ij')
    if indices:
        cropped_image = cv_img[indices]
    else:
        cropped_image = cv_img
    # Resize here !!
    resized_image = cv2.resize(cropped_image, (224,224))
    resized_image = np.array(resized_image,dtype=np.float64)
    
    #print(img.shape) #(224, 224, 3)
    img = np.expand_dims(resized_image, axis=0)
    #print(img.shape) #(1, 224, 224, 3)
    # 'RGB'->'BGR'
    img = img[:, :, :, ::-1]
    # Zero-center by mean pixel
    img[:, :, :, 0] -= 103.939
    img[:, :, :, 1] -= 116.779
    img[:, :, :, 2] -= 123.68
    #img[:, :, :, 0] -= 78.939
    #img[:, :, :, 1] -= 91.779
    #img[:, :, :, 2] -= 148.68
    #print(img.shape)
    #print(img)
    return img

In [9]:
#stop

In [10]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
# checkpoint
# http://machinelearningmastery.com/check-point-deep-learning-models-keras/
filen="VGG16D_1-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
filepath = os.path.join(weights_dir, filen)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')
# !!! TRAIN
#callbacks_list = [checkpoint, early]
#callbacks_list= [checkpoint]
callbacks_list=None

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_pk,
                                horizontal_flip=True,
                                vertical_flip=True,
                                 channel_shift_range=5)
        #rotation_range=20,
        #width_shift_range=0.1,
        #height_shift_range=0.1,
        #shear_range=0.1,
        #zoom_range=0.1,
        #horizontal_flip=True,
        #fill_mode='nearest')
test_datagen = ImageDataGenerator(preprocessing_function=preprocess)


#Arguments
#    x: input Numpy tensor, 4D.
#    data_format: data format of the image tensor.

In [12]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    classes = ['Type_1', 'Type_2', 'Type_3'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    classes = ['Type_1', 'Type_2', 'Type_3'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical')

Found 473 images belonging to 3 classes.
Found 362 images belonging to 3 classes.


In [13]:
# Use this for preliminary timing on the AWS p2 instance, so can estimate a good size for the sample directory files
import time
start_time = time.time()

In [ ]:
# !! SAMPLE
#class_weight_dict = utils.get_class_weight_dict([35,106,62])
# !! TRAIN
class_weight_dict = utils.get_class_weight_dict([250,781,450])

In [ ]:
h = vgg16d_1.fit_generator(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps, 
    class_weight = class_weight_dict,
    callbacks=callbacks_list,
    verbose=1)

Epoch 1/15
8/8 [==============================] - 199s - loss: 16.8839 - acc: 0.3575 - val_loss: 10.0196 - val_acc: 0.3674
Epoch 2/15
8/8 [==============================] - 130s - loss: 16.1175 - acc: 0.4068 - val_loss: 9.6649 - val_acc: 0.3757
Epoch 3/15
8/8 [==============================] - 119s - loss: 15.9601 - acc: 0.4294 - val_loss: 10.5211 - val_acc: 0.3260
Epoch 4/15
8/8 [==============================] - 105s - loss: 17.2920 - acc: 0.3893 - val_loss: 9.6887 - val_acc: 0.3867
Epoch 5/15
8/8 [==============================] - 107s - loss: 17.2381 - acc: 0.3614 - val_loss: 10.7020 - val_acc: 0.3232
Epoch 6/15
8/8 [==============================] - 119s - loss: 17.1632 - acc: 0.3769 - val_loss: 10.4873 - val_acc: 0.3398
Epoch 7/15
8/8 [==============================] - 111s - loss: 16.4184 - acc: 0.4343 - val_loss: 9.9683 - val_acc: 0.3674
Epoch 8/15
8/8 [==============================] - 114s - loss: 16.3407 - acc: 0.4292 - val_loss: 10.7823 - val_acc: 0.3149
Epoch 9/15
7/8 [===

In [ ]:
# Use this for preliminary timing on the AWS p2 instance, so can estimate a good size for the sample directory files
print("%f seconds" % (time.time() - start_time))
print('Total Images Processed ' , str(num_training_samples + num_validation_samples))
print('Epochs :', str(h.epoch[-1]+ 1))

In [ ]:
utils.plot_learning(h)

In [ ]:
vgg16d_1.optimizer.lr=1e-4 # Lift the learning rate
h = vgg16d_1.fit_generator(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_steps, 
    class_weight = class_weight_dict,
    callbacks=callbacks_list,
    verbose=1)

In [ ]:
utils.plot_learning(h)

In [ ]:
vgg16d_1.optimizer.lr=1e-5 # Seems to be just bouncing around, so put lr back to original
h = vgg16d_1.fit_generator(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_steps, 
    class_weight = class_weight_dict,
    callbacks=callbacks_list,
    verbose=1)
utils.plot_learning(h)

In [ ]:
vgg16d_1.optimizer.lr=1e-5 # 
h = vgg16d_1.fit_generator(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps, 
    class_weight = class_weight_dict,
    callbacks=callbacks_list,
    verbose=1)
utils.plot_learning(h)

In [ ]:
h = vgg16d_1.fit_generator(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps, 
    class_weight = class_weight_dict,
    callbacks=callbacks_list,
    verbose=1)
utils.plot_learning(h)

In [ ]:
h = vgg16d_1.fit_generator(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps, 
    class_weight = class_weight_dict,
    callbacks=callbacks_list,
    verbose=1)
utils.plot_learning(h)

In [ ]:
vgg16d_1.optimizer.lr=1e-6
h = vgg16d_1.fit_generator(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps, 
    class_weight = class_weight_dict,
    callbacks=callbacks_list,
    verbose=1)
utils.plot_learning(h)